# Custom Models and Training with TensorFlow

Up until now we've used only TensorFlow high-level API, `tf.keras`. In most cases you will only use `tf.keras`. But when you want want extra control over your model, cost function, optimization, etc then you have to know about TensorFlow's low-level API.

# A Quick tour of TensorFlow

TensorFlow is the most popular Machine Learning framework.

- Its core is similar to NumPy but with GPU.
- It supports distributed computing.
- It uses computational graphs which can be exported in any format. (with this you can use it with python or javascript)

TensorFlow offers many features such as preprocessing, model building, visualization, deployment, mathematics, etc.

TensorFlow's Python API overview:

```python
# High-level
tf.keras
tf.estimators

# Low-level
tf.nn
tf.losses
tf.metrices
tf.optimizers
tf.train
tf.initializers

# Autodiff
tf.GradientTape
tf.gradients()

# I/O and prerpcessing
tf.data
tf.image
tf.audio
tf.feature_column
tf.io
tf.queue
```

# Using TensorFlow like NumPy

TensorFlow uses *tensors*, which flow from operation to operation hence the name TensorFlow. A tensor is basically a multi-dimensional array. 

## Tensors and Operations

Creating tensor:

In [4]:
import tensorflow as tf

t = tf.constant([[1., 2., 3.], [4., 5., 6.]]) # matrix
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

Shape and type:

In [5]:
t.shape

TensorShape([2, 3])

In [6]:
t.dtype

tf.float32

Slicing like NumPy array:

In [7]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [10]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

Operations

In [11]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [12]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [13]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

## Tensors and NumPy

Tensors are friendly to NumPy arrays, they can be converted into each other:

In [14]:
import numpy as np

a = np.array([2., 4., 6.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 6.])>

In [15]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

You can even apply TensorFlow operations to NumPy array and vice-versa:

In [16]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 36.])>

In [17]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

> Type conversion is sensitive in TensorFlow so be careful when using different types.

## Variables

The `tf.Tensor` are immutable, you can't modify them. So, for weights we need `tf.Variable`:

In [18]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

They are pretty much same as `tf.Tensor`. Now, you can modify them using `assign()` method:

In [21]:
print(v.assign(2 * v))
print(v[0, 1].assign(42))
print(v[:, 2].assign([0., 1.]))
print(v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.]))

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 8., 84.,  0.],
       [32., 40.,  2.]], dtype=float32)>
<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 8., 42.,  0.],
       [32., 40.,  2.]], dtype=float32)>
<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 8., 42.,  0.],
       [32., 40.,  1.]], dtype=float32)>
<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [ 32.,  40., 200.]], dtype=float32)>


# Customizing Models and Training Algorithms

Let's start by customizing loss function.

## Custom Loss Functions

Let's make a huber cost function (which is a combination of both MSE and MAE):

```python
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)
```

Now you can use this when you compile the Keras model:

```python
model.compile(loss=huber_fn, optimizer='nadam')
model.fit(X_train, y_train, [...])
```

## Saving and Loading Models That contain Custom Components

Saving a model containing custom components is easy, as Keras saves the name of functions. Morever, when you load it you need to pass custom functions as a dictionary with name:

```python
model = keras.models.load_model('my_model.h5',
                               custom_objects={'huber_fn': huber_fn})
```

In this any error between -1 and 1 is considered to be small. But, if you want a different threshold and pass it to `huber_fn()`, then Keras will not save the threshold value. This can be solved by subclassing `keras.losses.Loss` class and by accessing `get_config()` function:

```python
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__()
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return (**base_config, 'threshold': self.threshold)
```

Now, you can use it during compiling with threshold. Now Keras saves it with configs, so when loading you don't need to give threshold value.